1. Wygenerowanie jednego rozwiązania ze ścieżką
2. Próba znalezienia pośredniego połączenia między dwoma połączeniami w rozwiazaniu
3. Jeśli to rozwiązanie polepsza wynik, to akceptujemy

In [4]:
from ai_data_eng.searching.graph import *
from ai_data_eng.searching.utils import *
from ai_data_eng.searching.searchning import * 
from ai_data_eng.searching.initialization import *
from ai_data_eng.searching.a_star_time_opt import a_star_time_opt_light, find_path_a_star_t
from ai_data_eng.searching.a_star_changes_opt import a_star_changes_opt_light, find_path_a_star_p, path_to_list_p
from ai_data_eng.searching.heuristics import WeightedAverageTimeHeuristic, ChangeHeuristic
from pathlib import Path

DATA_DIR = Path('../data')

def print_path(connections, print_to=None):
    for i, conn in enumerate(connections):
        print(
            f'({i}) {conn["start_stop"]} [{sec_to_time(conn["departure_sec"])}] --- {conn["line"]} ---> {conn["end_stop"]} [{sec_to_time(conn["arrival_sec"])}] [{conn.name}]',
            file=print_to)

In [5]:
connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv', 
                               usecols=['line', 'departure_time', 'arrival_time', 'start_stop',
       'end_stop', 'start_stop_lat', 'start_stop_lon', 'end_stop_lat',
       'end_stop_lon'])
g = Graph(connection_graph, partial(add_const_change_time, change_time=0))

C:\Users\julia\AppData\Local\Temp\ipykernel_36544\1491564076.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


In [43]:
from typing import Set
from queue import PriorityQueue

CRITERION = OptimizationType.TIME

def get_matched_stops(solution, visiting_stops: List[str]):
    visiting_stops = set(visiting_stops)
    matched_stops = 0
    for conn in solution:
        if conn['start_stop'] in visiting_stops:
            visiting_stops.remove(conn['start_stop'])
            matched_stops += 1
    return matched_stops, visiting_stops

def judge_t_solution(solution, visiting_stops: List[str]):
    conn_time = diff(solution[-1]['arrival_sec'], solution[-1]['departure_sec'])
    return conn_time / (get_matched_stops(solution, visiting_stops) + 0.01)    

def judge_p_solution(solution, visiting_stops: Set[str]):
    matched_stops = 0 
    line_changes = int(np.sum([1 for (c1, c2) in zip(solution[:-1], solution[1:]) if is_conn_change(c1, c2)]))
    
    for conn in solution:
        if conn['start_stop'] in visiting_stops:
            visiting_stops.remove(conn['start_stop'])
            matched_stops += 1
    return line_changes / (matched_stops + 0.01)  
    
def get_a_star(criterion: OptimizationType):
    if criterion == OptimizationType.TIME:
        return partial(find_path_a_star_t, graph=g, heuristic=WeightedAverageTimeHeuristic(),
                       cost_func=g.time_cost_between_conns, neighbours_gen=g.get_earliest_from_with_and_without_change)
    elif criterion == OptimizationType.CHANGES:
        return partial(find_path_a_star_p, graph=g, heuristic=ChangeHeuristic(),
                       cost_func=g.change_cost_between_conns, initialization_func=initialize_with_prev_conn, neighbours_gen=g.get_lines_from)


def get_connection_path(criterion: OptimizationType):
    if criterion ==  OptimizationType.TIME:
        def recreate(goal_index, came_from):
            return idxs_to_nodes(g, goal_index, came_from)
        return recreate
    elif criterion == OptimizationType.CHANGES:
        def recreate(goal, came_from):
            came_from_conn, stop_conn = came_from
            conns = path_to_list_p(goal, came_from_conn, stop_conn)
            connections = [g.conn_at_index(idx) for idx in conns]
            return connections
        return recreate


def naive_solution(criterion: OptimizationType, start_stop: str, visiting_stops: List[str], leave_hour: str):
    a_star = get_a_star(criterion)
    conn_path = get_connection_path(criterion)
    prev_stop = start_stop
    solution = []
    for stop in visiting_stops:
        goal, came_from, costs = a_star(start_stop=prev_stop, goal_stop=stop, leave_hour=leave_hour)
        subsol = conn_path(goal, came_from)
        solution += subsol
        prev_stop = stop
        # add change time
        leave_hour = sec_to_time(subsol[-1]['arrival_sec'])
        g.reset()
    goal, came_from, costs = a_star(start_stop=prev_stop, goal_stop=start_stop, leave_hour=leave_hour)
    solution += conn_path(goal, came_from)
    g.reset()
    return solution

In [18]:
sol1 = naive_solution(OptimizationType.TIME, 'PL. GRUNWALDZKI', ['most Grunwaldzki', 'Poczta Główna', 'Renoma'], '08:00:00')

In [19]:
print_path(sol1)

PL. GRUNWALDZKI [08:00:00] --- 146 ---> most Grunwaldzki [08:02:00] [849563]
most Grunwaldzki [08:02:00] --- 146 ---> Poczta Główna [08:04:00] [849564]
Poczta Główna [08:04:00] --- D ---> GALERIA DOMINIKAŃSKA [08:06:00] [14559]
GALERIA DOMINIKAŃSKA [08:06:00] --- D ---> Renoma [08:09:00] [14560]
Renoma [08:09:00] --- A ---> Arkady (Capitol) [08:11:00] [286]
Arkady (Capitol) [08:13:00] --- D ---> GALERIA DOMINIKAŃSKA [08:18:00] [23021]
GALERIA DOMINIKAŃSKA [08:18:00] --- D ---> Urząd Wojewódzki (Impart) [08:21:00] [23022]
Urząd Wojewódzki (Impart) [08:21:00] --- D ---> most Grunwaldzki [08:22:00] [23023]
most Grunwaldzki [08:22:00] --- D ---> PL. GRUNWALDZKI [08:24:00] [23024]


In [20]:
judge_t_solution(sol1, ['most Grunwaldzki', 'Poczta Główna', 'Renoma'])

39.8671096345515

In [39]:
vis_stops = ['Paprotna', 'Czajkowskiego', 'Ogród Botaniczny', 'Krucza', 'Renoma', 'Kępińska']
start_stop = 'PL. GRUNWALDZKI'
leave_hour = '08:00:00'

In [8]:
sol3 = naive_solution(OptimizationType.TIME, start_stop, vis_stops, leave_hour)

In [14]:
judge_t_solution(sol3, vis_stops)

29.950083194675543

In [9]:
print_path(sol3)

(0) PL. GRUNWALDZKI [08:00:00] --- D ---> Kochanowskiego [08:03:00] [22596]
(1) Kochanowskiego [08:03:00] --- D ---> Śniadeckich [08:05:00] [22597]
(2) Śniadeckich [08:05:00] --- D ---> Zacisze [08:06:00] [22598]
(3) Zacisze [08:06:00] --- D ---> Kwidzyńska [08:08:00] [22599]
(4) Kwidzyńska [08:08:00] --- 6 ---> Kętrzyńska [08:09:00] [135055]
(5) Kętrzyńska [08:09:00] --- 6 ---> KROMERA [08:11:00] [135056]
(6) KROMERA [08:13:00] --- 116 ---> Berenta [08:14:00] [556881]
(7) Berenta [08:14:00] --- 116 ---> Kasprowicza [08:15:00] [556882]
(8) Kasprowicza [08:15:00] --- 130 ---> Syrokomli [08:16:00] [711637]
(9) Syrokomli [08:16:00] --- 130 ---> Pola [08:17:00] [711638]
(10) Pola [08:17:00] --- 130 ---> Broniewskiego [08:18:00] [711639]
(11) Broniewskiego [08:18:00] --- 118 ---> Bałtycka [08:20:00] [567781]
(12) Bałtycka [08:20:00] --- 118 ---> Bezpieczna [08:21:00] [567782]
(13) Bezpieczna [08:21:00] --- 602 ---> Paprotna [08:22:00] [933891]
(14) Paprotna [08:25:00] --- 111 ---> Obornicka

In [17]:
g.conn_at_index(918837)

line                                            253
departure_time                             23:10:00
arrival_time                               23:11:00
start_stop                          PL. GRUNWALDZKI
end_stop          Kliniki - Politechnika Wrocławska
start_stop_lat                            51.111981
start_stop_lon                            17.064034
end_stop_lat                              51.109283
end_stop_lon                              17.066162
departure_sec                                 83400
arrival_sec                                   83460
Name: 918837, dtype: object

In [6]:
from ai_data_eng.searching.initialization import initialize_queue, initialize_with_prev_conn
# print_info = a_star_print_info(sec_to_time)
cost_so_far = {}
# if commuting A -> B, then this will be came_from_conn[B] = A so we can recreate the path
came_from_conn = {}
stop_conn = {}

prev_conn_idx = 278511

start_stop = 'most Grunwaldzki'
goal_stop = 'Urząd Wojewódzki (Impart)'
dep_time = time_to_normalized_sec('08:36:00')
frontier = initialize_with_prev_conn(prev_conn_idx, g, cost_so_far, came_from_conn, stop_conn, start_stop, dep_time)

In [30]:
cost_so_far

{'most Grunwaldzki': 0}

In [31]:
came_from_conn

{-1: None, 278511: None}

In [32]:
stop_conn

{'most Grunwaldzki': 278511}

In [33]:
g.stop_as_tuple(g.rename_stop(g.conn_at_index(918837)))

('Kliniki - Politechnika Wrocławska', 51.109283, 17.066162)

In [34]:
prev_conn = g.conn_at_index(prev_conn_idx)

278512, 97887, 233196


In [35]:
g.conn_graph.loc[[240344, 97887, 278512]]

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
240344,13,08:36:00,08:38:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,30960,31080
97887,4,08:37:00,08:39:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,31020,31140
278512,16,08:36:00,08:38:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,30960,31080


In [36]:
g.conn_graph.loc[(g.conn_graph['start_stop'] == start_stop) & (g.conn_graph['end_stop'] == goal_stop)].sort_values(by=['departure_sec'])

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
277379,16,04:57:00,04:59:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,17820,17940
107188,4,05:01:00,05:03:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,18060,18180
271778,16,05:12:00,05:14:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,18720,18840
107549,4,05:16:00,05:18:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,18960,19080
100605,4,05:17:00,05:19:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,19020,19140
...,...,...,...,...,...,...,...,...,...,...,...
219138,12,23:19:00,23:20:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,83940,84000
320498,19,23:24:00,23:25:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,84240,84300
231995,13,23:27:00,23:28:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,84420,84480
273793,16,23:32:00,23:33:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,84720,84780


In [37]:
for next_conn in g.get_lines_from(prev_conn).itertuples():
    # cost of commuting start --> current and current --> next
    next_stop = (next_conn.end_stop, next_conn.end_stop_lat, next_conn.end_stop_lon)
    print(next_conn.start_stop, next_conn.end_stop, next_conn.line, g.change_cost_between_conns(next_conn=next_conn, prev_conn=prev_conn))

most Grunwaldzki PL. GRUNWALDZKI 16 1
most Grunwaldzki Urząd Wojewódzki (Impart) 13 1
most Grunwaldzki PL. GRUNWALDZKI 12 1
most Grunwaldzki Urząd Wojewódzki (Impart) 16 0
most Grunwaldzki PL. GRUNWALDZKI D 1
most Grunwaldzki Urząd Wojewódzki (Impart) 4 1
most Grunwaldzki Poczta Główna 145 1
most Grunwaldzki PL. GRUNWALDZKI 13 1
most Grunwaldzki Poczta Główna D 1
most Grunwaldzki PL. GRUNWALDZKI 145 1
most Grunwaldzki Poczta Główna 149 1
most Grunwaldzki Urząd Wojewódzki (Impart) 12 1
most Grunwaldzki PL. GRUNWALDZKI 4 1
most Grunwaldzki Poczta Główna 146 1
most Grunwaldzki PL. GRUNWALDZKI 146 1
most Grunwaldzki PL. GRUNWALDZKI 149 1
most Grunwaldzki Urząd Wojewódzki (Impart) 19 1
most Grunwaldzki PL. GRUNWALDZKI 19 1
most Grunwaldzki Poczta Główna 241 1
most Grunwaldzki Urząd Wojewódzki (Muzeum Narodowe) 240 1
most Grunwaldzki PL. GRUNWALDZKI 241 1


In [10]:
print_path(sol3)

(0) PL. GRUNWALDZKI [08:00:00] --- D ---> Kochanowskiego [08:03:00] [22596]
(1) Kochanowskiego [08:03:00] --- D ---> Śniadeckich [08:05:00] [22597]
(2) Śniadeckich [08:05:00] --- D ---> Zacisze [08:06:00] [22598]
(3) Zacisze [08:06:00] --- D ---> Kwidzyńska [08:08:00] [22599]
(4) Kwidzyńska [08:08:00] --- 6 ---> Kętrzyńska [08:09:00] [135055]
(5) Kętrzyńska [08:09:00] --- 6 ---> KROMERA [08:11:00] [135056]
(6) KROMERA [08:13:00] --- 116 ---> Berenta [08:14:00] [556881]
(7) Berenta [08:14:00] --- 116 ---> Kasprowicza [08:15:00] [556882]
(8) Kasprowicza [08:15:00] --- 130 ---> Syrokomli [08:16:00] [711637]
(9) Syrokomli [08:16:00] --- 130 ---> Pola [08:17:00] [711638]
(10) Pola [08:17:00] --- 130 ---> Broniewskiego [08:18:00] [711639]
(11) Broniewskiego [08:18:00] --- 118 ---> Bałtycka [08:20:00] [567781]
(12) Bałtycka [08:20:00] --- 118 ---> Bezpieczna [08:21:00] [567782]
(13) Bezpieczna [08:21:00] --- 602 ---> Paprotna [08:22:00] [933891]
(14) Paprotna [08:25:00] --- 111 ---> Obornicka

In [11]:
a_star = get_a_star(OptimizationType.TIME)
conn_path = get_connection_path(OptimizationType.TIME)

def conn_e_i_e_j(connections, i, j):
    goal_index, came_from, _ = a_star(start_stop=connections[i].start_stop, goal_stop=connections[j].end_stop, leave_hour=sec_to_time(connections[i - 1].arrival_sec), prev_conn_idx=connections[i - 1].name if i > 0 else None)
    g.reset()
    return conn_path(goal_index, came_from)

def conn_e_j_e_i(i_conn, j_conn):
    goal_index, came_from, _ = a_star(start_stop=j_conn.end_stop, goal_stop=i_conn.end_stop, leave_hour=sec_to_time(j_conn.arrival_sec), prev_conn_idx=j_conn.name)
    g.reset()
    return conn_path(goal_index, came_from)



def swap_conns(connections, i, j, criterion: OptimizationType):
    # start stop of i --- > end stop of j
    # end stop of i ---> start stop of j + 1
    # end stop of j ---> start stop of i + 1 
    a_star = get_a_star(criterion)
    conn_path = get_connection_path(criterion)
    conn_s_i_e_j = a_star(start_stop=connections[i].start_stop, goal_stop=connections[j].end_stop, leave_hour=sec_to_time(connections[i].departure_sec), prev_conn_idx=connections[i - 1] if i > 0 else None)
    

In [46]:
sol_a = conn_e_i_e_j(sol3, 0, 30)

In [13]:
print_path(sol_a)

(0) PL. GRUNWALDZKI [10:30:00] --- 16 ---> Piastowska [10:32:00] [279406]
(1) Piastowska [10:32:00] --- 16 ---> Prusa [10:34:00] [279407]
(2) Prusa [10:34:00] --- 16 ---> Wyszyńskiego [10:35:00] [279408]
(3) Wyszyńskiego [10:35:00] --- 16 ---> Nowowiejska [10:37:00] [279409]
(4) Nowowiejska [10:38:00] --- 23 ---> Daszyńskiego [10:39:00] [383047]
(5) Daszyńskiego [10:40:00] --- 111 ---> Słonimskiego [10:41:00] [514575]
(6) Słonimskiego [10:41:00] --- 111 ---> Zakładowa [10:42:00] [514576]
(7) Zakładowa [10:42:00] --- 111 ---> Broniewskiego [10:45:00] [514577]
(8) Broniewskiego [10:45:00] --- 111 ---> Bałtycka [10:47:00] [514578]
(9) Bałtycka [10:47:00] --- 111 ---> Bezpieczna [10:49:00] [514579]


In [14]:
sol_b = conn_e_j_e_i(sol3[0], sol_a[-1])

In [15]:
print_path(sol_b)

(0) Bezpieczna [10:51:00] --- 105 ---> Bałtycka (szkoła) [10:52:00] [453526]
(1) Bałtycka (szkoła) [10:52:00] --- 105 ---> Bałtycka [10:53:00] [453527]
(2) Bałtycka [10:53:00] --- 105 ---> Broniewskiego [10:55:00] [453528]
(3) Broniewskiego [10:57:00] --- 111 ---> Zakładowa [10:59:00] [504670]
(4) Zakładowa [10:59:00] --- 111 ---> Słonimskiego [11:00:00] [504671]
(5) Słonimskiego [11:00:00] --- 111 ---> Daszyńskiego [11:02:00] [504672]
(6) Daszyńskiego [11:02:00] --- 111 ---> Nowowiejska [11:04:00] [504673]
(7) Nowowiejska [11:04:00] --- 16 ---> Wyszyńskiego [11:06:00] [273249]
(8) Wyszyńskiego [11:06:00] --- 16 ---> Prusa [11:07:00] [273250]
(9) Prusa [11:07:00] --- 16 ---> Piastowska [11:09:00] [273251]
(10) Piastowska [11:11:00] --- 9 ---> Grunwaldzka [11:12:00] [173584]
(11) Grunwaldzka [11:12:00] --- 9 ---> Kochanowskiego [11:14:00] [173585]


In [16]:
print_path(conn_e_j_e_i(sol3[16], sol_b[-1]))

(0) Kochanowskiego [11:15:00] --- 17 ---> Grunwaldzka [11:16:00] [286807]
(1) Grunwaldzka [11:16:00] --- 17 ---> Piastowska [11:17:00] [286808]
(2) Piastowska [11:17:00] --- 16 ---> Prusa [11:19:00] [278670]
(3) Prusa [11:19:00] --- 16 ---> Wyszyńskiego [11:20:00] [278671]
(4) Wyszyńskiego [11:20:00] --- 16 ---> Nowowiejska [11:22:00] [278672]
(5) Nowowiejska [11:24:00] --- 6 ---> Daszyńskiego [11:25:00] [133852]
(6) Daszyńskiego [11:25:00] --- 111 ---> Słonimskiego [11:26:00] [513702]
(7) Słonimskiego [11:26:00] --- 111 ---> Zakładowa [11:27:00] [513703]
(8) Zakładowa [11:27:00] --- 111 ---> Broniewskiego [11:30:00] [513704]
(9) Broniewskiego [11:30:00] --- 111 ---> Bałtycka [11:32:00] [513705]


In [17]:
g.conn_at_index(513705)

line                        111
departure_time         11:30:00
arrival_time           11:32:00
start_stop        Broniewskiego
end_stop               Bałtycka
start_stop_lat        51.135904
start_stop_lon        17.035327
end_stop_lat          51.138282
end_stop_lon          17.029374
departure_sec             41400
arrival_sec               41520
Name: 513705, dtype: object

In [18]:
g.conn_at_index(32289)

line                          K
departure_time         08:29:00
arrival_time           08:31:00
start_stop             Bałtycka
end_stop          Broniewskiego
start_stop_lat        51.136632
start_stop_lon        17.030617
end_stop_lat          51.136343
end_stop_lon          17.036687
departure_sec             30540
arrival_sec               30660
Name: 32289, dtype: object

In [21]:
new_conn = g.get_earliest_from_to(g.conn_at_index(513705), g.conn_at_index(32289)).iloc[0]

In [23]:
new_conn

line                        118
departure_time         11:34:00
arrival_time           11:36:00
start_stop             Bałtycka
end_stop          Broniewskiego
start_stop_lat        51.136632
start_stop_lon        17.030617
end_stop_lat          51.135851
end_stop_lon          17.037383
departure_sec             41640
arrival_sec               41760
Name: 570614, dtype: object

In [22]:
g.get_earliest_from_to(new_conn, g.conn_at_index(871098))

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
570615,118,11:36:00,11:37:00,Broniewskiego,Pola,51.135851,17.037383,51.136749,17.044698,41760,41820


In [54]:
def swap_connections(connections, i, j):
    solution = connections[:i]
    sol_a = conn_e_i_e_j(sol3, i, j)
    sol_b = conn_e_j_e_i(connections[i], sol_a[-1])
    sol_c = conn_e_j_e_i(sol3[j + 1], sol_b[-1])
    solution += sol_a + sol_b + sol_c
    prev_conn = sol_c[-1]
    for conn in connections[j:]:
        new_conn = g.get_earliest_from_to(prev_conn, conn).iloc[0]
        solution.append(new_conn)
        prev_conn = new_conn
    return solution

In [55]:
swapped_sol3 = swap_connections(sol3, 10, 20)

In [56]:
vis_stops

['Paprotna',
 'Czajkowskiego',
 'Ogród Botaniczny',
 'Krucza',
 'Renoma',
 'Kępińska']

In [57]:
get_matched_stops(swapped_sol3, vis_stops)

(5, {'Paprotna'})

In [58]:
get_matched_stops(sol3, vis_stops)

(6, set())

In [60]:
print_path(sol3)

(0) PL. GRUNWALDZKI [08:00:00] --- D ---> Kochanowskiego [08:03:00] [22596]
(1) Kochanowskiego [08:03:00] --- D ---> Śniadeckich [08:05:00] [22597]
(2) Śniadeckich [08:05:00] --- D ---> Zacisze [08:06:00] [22598]
(3) Zacisze [08:06:00] --- D ---> Kwidzyńska [08:08:00] [22599]
(4) Kwidzyńska [08:08:00] --- 6 ---> Kętrzyńska [08:09:00] [135055]
(5) Kętrzyńska [08:09:00] --- 6 ---> KROMERA [08:11:00] [135056]
(6) KROMERA [08:13:00] --- 116 ---> Berenta [08:14:00] [556881]
(7) Berenta [08:14:00] --- 116 ---> Kasprowicza [08:15:00] [556882]
(8) Kasprowicza [08:15:00] --- 130 ---> Syrokomli [08:16:00] [711637]
(9) Syrokomli [08:16:00] --- 130 ---> Pola [08:17:00] [711638]
(10) Pola [08:17:00] --- 130 ---> Broniewskiego [08:18:00] [711639]
(11) Broniewskiego [08:18:00] --- 118 ---> Bałtycka [08:20:00] [567781]
(12) Bałtycka [08:20:00] --- 118 ---> Bezpieczna [08:21:00] [567782]
(13) Bezpieczna [08:21:00] --- 602 ---> Paprotna [08:22:00] [933891]
(14) Paprotna [08:25:00] --- 111 ---> Obornicka

In [59]:
print_path(swapped_sol3)

(0) PL. GRUNWALDZKI [08:00:00] --- D ---> Kochanowskiego [08:03:00] [22596]
(1) Kochanowskiego [08:03:00] --- D ---> Śniadeckich [08:05:00] [22597]
(2) Śniadeckich [08:05:00] --- D ---> Zacisze [08:06:00] [22598]
(3) Zacisze [08:06:00] --- D ---> Kwidzyńska [08:08:00] [22599]
(4) Kwidzyńska [08:08:00] --- 6 ---> Kętrzyńska [08:09:00] [135055]
(5) Kętrzyńska [08:09:00] --- 6 ---> KROMERA [08:11:00] [135056]
(6) KROMERA [08:13:00] --- 116 ---> Berenta [08:14:00] [556881]
(7) Berenta [08:14:00] --- 116 ---> Kasprowicza [08:15:00] [556882]
(8) Kasprowicza [08:15:00] --- 130 ---> Syrokomli [08:16:00] [711637]
(9) Syrokomli [08:16:00] --- 130 ---> Pola [08:17:00] [711638]
(10) Pola [08:17:00] --- 130 ---> Syrokomli [08:18:00] [715449]
(11) Syrokomli [08:18:00] --- 130 ---> Kasprowicza [08:19:00] [715450]
(12) Kasprowicza [08:19:00] --- 130 ---> pl. Daniłowskiego [08:21:00] [715451]
(13) pl. Daniłowskiego [08:21:00] --- 130 ---> Berenta [08:22:00] [715452]
(14) Berenta [08:23:00] --- 116 --->

In [31]:
judge_t_solution(sol3, vis_stops)

19.966722129783694

In [32]:
judge_t_solution(swapped_sol3, vis_stops)

12000.0

In [16]:
print_path(conn_s_i_e_j(sol3, 24, 13))

(0) Czajkowskiego [08:42:00] --- 116 ---> Koszarowa [08:44:00] [558576]
(1) Koszarowa [08:47:00] --- 116 ---> KOSZAROWA (Szpital) [08:48:00] [559320]
(2) KOSZAROWA (Szpital) [08:48:00] --- 116 ---> pl. Daniłowskiego [08:51:00] [559321]
(3) pl. Daniłowskiego [08:51:00] --- 116 ---> Kasprowicza [08:52:00] [559322]
(4) Kasprowicza [08:54:00] --- 130 ---> Syrokomli [08:55:00] [712170]
(5) Syrokomli [08:55:00] --- 130 ---> Pola [08:56:00] [712171]
(6) Pola [08:56:00] --- 130 ---> Broniewskiego [08:57:00] [712172]
(7) Broniewskiego [09:03:00] --- 129 ---> Bałtycka [09:05:00] [697789]
(8) Bałtycka [09:05:00] --- 111 ---> Bezpieczna [09:07:00] [512560]
(9) Bezpieczna [09:07:00] --- 111 ---> Paprotna [09:08:00] [512561]


In [15]:
print_path(conn_s_i_e_j(sol3, 14, 31))

(0) Paprotna [08:25:00] --- 111 ---> Obornicka (Wołowska) [08:26:00] [512081]
(1) Obornicka (Wołowska) [08:26:00] --- 111 ---> Bezpieczna [08:27:00] [512082]
(2) Bezpieczna [08:27:00] --- K ---> Bałtycka [08:29:00] [32288]
(3) Bałtycka [08:29:00] --- K ---> Broniewskiego [08:31:00] [32289]
(4) Broniewskiego [08:32:00] --- 149 ---> Pola [08:33:00] [871098]
(5) Pola [08:33:00] --- 149 ---> Syrokomli [08:34:00] [871099]
(6) Syrokomli [08:34:00] --- 149 ---> Berenta [08:36:00] [871100]
(7) Berenta [08:36:00] --- 149 ---> KROMERA [08:39:00] [871101]
(8) KROMERA [08:39:00] --- 128 ---> Mosty Warszawskie [08:41:00] [686148]
(9) Mosty Warszawskie [08:41:00] --- 128 ---> Wyszyńskiego [08:43:00] [686149]
(10) Wyszyńskiego [08:43:00] --- 128 ---> Ogród Botaniczny [08:46:00] [686150]


In [ ]:
def is_neighbour(dep_time, new_solution):
    return assert_connection_path(dep_time, new_solution)

def naive_search(solution, leave_hour, start_stop, visiting_stops: List[str]):
    prev_stop = start_stop
    new_solution = []
    for stop in visiting_stops:
        _, sub_sol = a_star_time_opt(prev_stop, stop, leave_hour, WeightedAverageTimeHeuristic())
        new_solution = new_solution + sub_sol
        prev_stop = 

def replace_connection(solution, dep_time, stop_from, stop_to):
    
    is_neigh = False
    while not is_neigh:
        graph, travel_solution = a_star_time_opt(stop_from, stop_to, dep_time, WeightedAverageTimeHeuristic())